# Project 3: Recurrent Neural Networks

<div class="alert alert-block alert-info">
    Welcome to Project 3: Recurrent Neural Networks!<br>
    <br>
    In this project you will work with a basic RNN architecture for text generation, namely Sequence to Seqeuence Models. The name points out that these models take a sequence as input and return another sequence as output (see Fig. 1).<br>
    <br>
    Sequence to Vector Models are similar, but return only a single output in the final time step (see Fig. 2). The latter architecture is, for example, suitable for sequence classification.
</div>
<br>
<table style="width:100%">
    <tr>
        <th><img src="rnn_seq_2_seq.png?666" alt="" style="width: 475px;"></th>
        <th><img src="rnn_seq_2_vec.png?666" alt="" style="width: 475px;"></th>
    </tr>
    <tr>
        <th>Fig. 1: Sequence to Sequence RNN with one hidden layer.</th>
        <th>Fig. 2: Sequence to Vector RNN with one hidden layer.</th>
    </tr>
</table>

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

2023-01-16 23:46:27.618007: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7856400151119191667
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2475294720
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4390755526817733996
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


2023-01-16 23:49:37.117088: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-16 23:49:37.159358: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-16 23:49:37.244162: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-16 23:49:37.244396: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so retur

In [3]:
tf.test.gpu_device_name()

2023-01-16 23:49:48.247433: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-16 23:49:48.247965: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-16 23:49:48.248366: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-16 23:49:48.248975: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-16 23:49:48.249332: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

'/device:GPU:0'

<div class="alert alert-block alert-info">
    Read the text files <font face='courier'>hamlet_1.txt</font>, <font face='courier'>hamlet_2.txt</font> and <font face='courier'>hamlet_3.txt</font>.<br>
    <br>
    Store their content in variables <font face='courier'>hamlet_1_text</font>, <font face='courier'>hamlet_2_text</font> and <font face='courier'>hamlet_3_text</font>, respectively.<br>
    <br>
    Be aware that all files are UTF-8 encoded. Maybe you find <a href='https://docs.python.org/3/tutorial/inputoutput.html#reading-and-writing-files'>this link</a> helpful.
</div>

In [5]:
# YOUR CODE
with open('hamlet_1.txt', encoding="utf-8") as f:
    hamlet_1_text = f.read()
with open('hamlet_2.txt', encoding="utf-8") as f:
    hamlet_2_text = f.read()
with open('hamlet_3.txt', encoding="utf-8") as f:
    hamlet_3_text = f.read()
    
f.close()

<div class="alert alert-block alert-info">
    Print the first 325 characters of <font face='courier'>hamlet_1_text</font>.
</div>

In [4]:
# YOUR CODE
print(hamlet_1_text[0:325])

The Tragedie of Hamlet

Actus Primus. Scoena Prima.

Enter Barnardo and Francisco two Centinels.

 Barnardo. Who's there?
Fran. Nay answer me: Stand & vnfold
your selfe

 Bar. Long liue the King

 Fran. Barnardo?
Bar. He

 Fran. You come most carefully vpon your houre

 Bar. 'Tis now strook twelue, get thee to bed Francisco


**Checkpoint:** You should see the following output:

```
The Tragedie of Hamlet

Actus Primus. Scoena Prima.

Enter Barnardo and Francisco two Centinels.

 Barnardo. Who's there?
Fran. Nay answer me: Stand & vnfold
your selfe

 Bar. Long liue the King

 Fran. Barnardo?
Bar. He

 Fran. You come most carefully vpon your houre

 Bar. 'Tis now strook twelue, get thee to bed Francisco

```

<div class="alert alert-block alert-info">
    Instantiate an object <font face='courier'>tokenizer</font> of type <font face='courier'>tf.keras.preprocessing.text.Tokenzier</font>.<br>
    <br>
    Submit the argument <font face='courier'>char_level=True</font> to the constructor.<br>
    <br>
    Helpful information can be found <a href='https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer'>here</a>.
</div>

In [5]:
# YOUR CODE
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level='True')

<div class="alert alert-block alert-info">
    The object <font face='courier'>tokenizer</font> will ultimately be used to encode the strings <font face='courier'>hamlet_1_text</font>, <font face='courier'>hamlet_2_text</font> and <font face='courier'>hamlet_3_text</font> as integer sequences in which each single number represents a specific character.<br>
    <br>
    Call the method <font face='courier'>fit_on_texts</font> of <font face='courier'>tokenizer</font>.<br>
    <br>
    In doing so, pass a list that contains exactly the strings <font face='courier'>hamlet_1_text</font>, <font face='courier'>hamlet_2_text</font> and <font face='courier'>hamlet_3_text</font>.
</div>

In [6]:
# YOUR CODE
data = hamlet_1_text + hamlet_2_text + hamlet_3_text 
tokenizer.fit_on_texts(data)

<div class="alert alert-block alert-info">
    The above call creates a vocabulary in <font face='courier'>tokenizer</font>. This vocabulary assigns a positive integer to each unique character in the texts that were passed to the <font face='courier'>fit_on_texts</font> method.<br>
    <br>
    Display the attribute <font face='courier'>word_index</font> of <font face='courier'>tokenizer</font> which contains the created vocabulary.
</div>

In [7]:
# YOUR CODE
index = tokenizer.word_index
print(tokenizer.word_index)

{' ': 1, 'e': 2, 't': 3, 'o': 4, 'a': 5, 'h': 6, 'i': 7, 'n': 8, 's': 9, 'r': 10, 'l': 11, '\n': 12, 'u': 13, 'd': 14, 'm': 15, 'y': 16, ',': 17, 'w': 18, 'f': 19, 'c': 20, 'g': 21, '.': 22, 'p': 23, 'b': 24, 'k': 25, "'": 26, ':': 27, 'v': 28, '?': 29, ';': 30, 'q': 31, 'x': 32, '-': 33, 'z': 34, '(': 35, ')': 36, '&': 37, '!': 38, '[': 39, ']': 40, '1': 41, 'j': 42}


**Checkpoint:** You should have got the following output:

```
{' ': 1, 'e': 2, 't': 3, 'o': 4, 'a': 5, 'h': 6, 'i': 7, 'n': 8, 's': 9, 'r': 10, 'l': 11, '\n': 12, 'u': 13, 'd': 14, 'm': 15, 'y': 16, ',': 17, 'w': 18, 'f': 19, 'c': 20, 'g': 21, '.': 22, 'p': 23, 'b': 24, 'k': 25, "'": 26, ':': 27, 'v': 28, '?': 29, ';': 30, 'q': 31, 'x': 32, '-': 33, 'z': 34, '(': 35, ')': 36, '&': 37, '!': 38, '[': 39, ']': 40, '1': 41, 'j': 42}

```

<div class="alert alert-block alert-info">
    Compute the length of the vocabulary, store it in the variable <font face='courier'>max_id</font> and display this variable.
</div>

In [8]:
# YOUR CODE
max_id = len(index)

<div class="alert alert-block alert-info">
    Now, use the method <font face='courier'>texts_to_sequences</font> of <font face='courier'>tokenzier</font> to encode <font face='courier'>hamlet_1_text</font>, <font face='courier'>hamlet_2_text</font> and <font face='courier'>hamlet_3_text</font>.<br>
    <br>
    Store the respective coded strings in <font face='courier'>hamlet_1_encoded</font>, <font face='courier'>hamlet_2_encoded</font> and <font face='courier'>hamlet_3_encoded</font>.<br>
    <br>
    Convert all three lists to the format <font face='courier'>numpy.array</font> and subtract <font face='courier'>1</font> from all entries so that all values range between <font face='courier'>0</font> and <font face='courier'>max_id - 1</font> afterwards.
</div>

In [9]:
# YOUR CODE
hamlet_1_encoded = tokenizer.texts_to_sequences(hamlet_1_text)
hamlet_2_encoded = tokenizer.texts_to_sequences(hamlet_2_text)
hamlet_3_encoded = tokenizer.texts_to_sequences(hamlet_3_text)

hamlet_1_encoded = np.hstack(hamlet_1_encoded)
hamlet_1_encoded = hamlet_1_encoded-1
hamlet_2_encoded = np.hstack(hamlet_2_encoded)
hamlet_2_encoded = hamlet_2_encoded-1
hamlet_3_encoded = np.hstack(hamlet_3_encoded)
hamlet_3_encoded = hamlet_3_encoded-1

<div class="alert alert-block alert-info">
    Display the first <font face='courier'>325</font> entries of <font face='courier'>hamlet_1_encoded</font>.
    <a href=''></a>
</div>

In [10]:
# YOUR CODE
print(hamlet_1_encoded[0:325])

[ 2  5  1  0  2  9  4 20  1 13  6  1  0  3 18  0  5  4 14 10  1  2 11 11
  4 19  2 12  8  0 22  9  6 14 12  8 21  0  8 19  3  1  7  4  0 22  9  6
 14  4 21 11 11  1  7  2  1  9  0 23  4  9  7  4  9 13  3  0  4  7 13  0
 18  9  4  7 19  6  8 19  3  0  2 17  3  0 19  1  7  2  6  7  1 10  8 21
 11 11  0 23  4  9  7  4  9 13  3 21  0 17  5  3 25  8  0  2  5  1  9  1
 28 11 18  9  4  7 21  0  7  4 15  0  4  7  8 17  1  9  0 14  1 26  0  8
  2  4  7 13  0 36  0 27  7 18  3 10 13 11 15  3 12  9  0  8  1 10 18  1
 11 11  0 23  4  9 21  0 10  3  7 20  0 10  6 12  1  0  2  5  1  0 24  6
  7 20 11 11  0 18  9  4  7 21  0 23  4  9  7  4  9 13  3 28 11 23  4  9
 21  0  5  1 11 11  0 18  9  4  7 21  0 15  3 12  0 19  3 14  1  0 14  3
  8  2  0 19  4  9  1 18 12 10 10 15  0 27 22  3  7  0 15  3 12  9  0  5
  3 12  9  1 11 11  0 23  4  9 21  0 25  2  6  8  0  7  3 17  0  8  2  9
  3  3 24  0  2 17  1 10 12  1 16  0 20  1  2  0  2  5  1  1  0  2  3  0
 23  1 13  0 18  9  4  7 19  6  8 19  3]


**Checkpoint:** Your output should be as follows:

```
[ 2  5  1  0  2  9  4 20  1 13  6  1  0  3 18  0  5  4 14 10  1  2 11 11
  4 19  2 12  8  0 22  9  6 14 12  8 21  0  8 19  3  1  7  4  0 22  9  6
 14  4 21 11 11  1  7  2  1  9  0 23  4  9  7  4  9 13  3  0  4  7 13  0
 18  9  4  7 19  6  8 19  3  0  2 17  3  0 19  1  7  2  6  7  1 10  8 21
 11 11  0 23  4  9  7  4  9 13  3 21  0 17  5  3 25  8  0  2  5  1  9  1
 28 11 18  9  4  7 21  0  7  4 15  0  4  7  8 17  1  9  0 14  1 26  0  8
  2  4  7 13  0 36  0 27  7 18  3 10 13 11 15  3 12  9  0  8  1 10 18  1
 11 11  0 23  4  9 21  0 10  3  7 20  0 10  6 12  1  0  2  5  1  0 24  6
  7 20 11 11  0 18  9  4  7 21  0 23  4  9  7  4  9 13  3 28 11 23  4  9
 21  0  5  1 11 11  0 18  9  4  7 21  0 15  3 12  0 19  3 14  1  0 14  3
  8  2  0 19  4  9  1 18 12 10 10 15  0 27 22  3  7  0 15  3 12  9  0  5
  3 12  9  1 11 11  0 23  4  9 21  0 25  2  6  8  0  7  3 17  0  8  2  9
  3  3 24  0  2 17  1 10 12  1 16  0 20  1  2  0  2  5  1  1  0  2  3  0
 23  1 13  0 18  9  4  7 19  6  8 19  3]
```

<div class="alert alert-block alert-info">
    The original texts can be recovered via the method <font face='courier'>sequences_to_texts</font> of <font face='courier'>tokenizer</font>.<br>
    <br>
    Texts recovered in this way are all lower case and each original character is followed by a blank space.<br>
    <br>
    Apply <font face='courier'>sequences_to_texts</font> to <font face='courier'>hamlet_1_encoded + 1</font> and store the result in <font face='courier'>hamlet_1_decoded</font>.<br>
    <br>
    Afterfwards, display the first <font face='courier'>649</font> characters of <font face='courier'>hamlet_1_decoded</font>.
</div>

In [11]:
# YOUR CODE
hamlet_1_encoded = np.vstack(hamlet_1_encoded)
hamlet_1_encoded = hamlet_1_encoded + 1
hamlet_1_decoded = tokenizer.sequences_to_texts(hamlet_1_encoded)
print(' '.join(hamlet_1_decoded)[:649])

t h e   t r a g e d i e   o f   h a m l e t 
 
 a c t u s   p r i m u s .   s c o e n a   p r i m a . 
 
 e n t e r   b a r n a r d o   a n d   f r a n c i s c o   t w o   c e n t i n e l s . 
 
   b a r n a r d o .   w h o ' s   t h e r e ? 
 f r a n .   n a y   a n s w e r   m e :   s t a n d   &   v n f o l d 
 y o u r   s e l f e 
 
   b a r .   l o n g   l i u e   t h e   k i n g 
 
   f r a n .   b a r n a r d o ? 
 b a r .   h e 
 
   f r a n .   y o u   c o m e   m o s t   c a r e f u l l y   v p o n   y o u r   h o u r e 
 
   b a r .   ' t i s   n o w   s t r o o k   t w e l u e ,   g e t   t h e e   t o   b e d   f r a n c i s c o


**Checkpoint:** Your output should look like this:

```
t h e   t r a g e d i e   o f   h a m l e t 
 
 a c t u s   p r i m u s .   s c o e n a   p r i m a . 
 
 e n t e r   b a r n a r d o   a n d   f r a n c i s c o   t w o   c e n t i n e l s . 
 
   b a r n a r d o .   w h o ' s   t h e r e ? 
 f r a n .   n a y   a n s w e r   m e :   s t a n d   &   v n f o l d 
 y o u r   s e l f e 
 
   b a r .   l o n g   l i u e   t h e   k i n g 
 
   f r a n .   b a r n a r d o ? 
 b a r .   h e 
 
   f r a n .   y o u   c o m e   m o s t   c a r e f u l l y   v p o n   y o u r   h o u r e 
 
   b a r .   ' t i s   n o w   s t r o o k   t w e l u e ,   g e t   t h e e   t o   b e d   f r a n c i s c o
```

<div class="alert alert-block alert-info">
    Create three objects <font face='courier'>hamlet_1_dataset</font>, <font face='courier'>hamlet_2_dataset</font> and <font face='courier'>hamlet_3_dataset</font> of type <font face='courier'>tf.data.dataset</font> by applying the method<br>
    <br>
    <font face='courier'>tf.data.Dataset.from_tensor_slices</font> (see <a href='https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices'>this link</a>) to <font face='courier'>hamlet_1_encoded</font>, <font face='courier'>hamlet_2_encoded</font> and <font face='courier'>hamlet_3_encoded</font>.
</div>

In [12]:
# YOUR CODE
hamlet_1_encoded = np.hstack(hamlet_1_encoded)
hamlet_1_encoded = hamlet_1_encoded-1

hamlet_1_dataset = tf.data.Dataset.from_tensor_slices(hamlet_1_encoded)
hamlet_2_dataset = tf.data.Dataset.from_tensor_slices(hamlet_2_encoded)
hamlet_3_dataset = tf.data.Dataset.from_tensor_slices(hamlet_3_encoded)

<div class="alert alert-block alert-info">
    Display the first ten elements of <font face='courier'>hamlet_1_dataset</font>.
</div>

In [13]:
# YOUR CODE
for element in hamlet_1_dataset.take(10):
    print(element)

tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(20, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


**Checkpoint:** You should have produced the following output:

```
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(20, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)
```

<div class="alert alert-block alert-info">
    As you can see, each item of <font face='courier'>hamlet_1_dataset</font> is an integer tensor including one single value.<br>
    <br>
    Accordingly, <font face='courier'>hamlet_1_dataset</font> has <font face='courier'>len(hamlet_1_encoded)</font> elements in total (the same applies in case of the other two datasets).<br>
    <br>
    In the following, you will train a recurrent neural network which gets a coded string of length <font face='courier'>T = 100</font> and predicts subsequent characters in all time steps.<br>
    <br>
    Accordingly, we will first transform our three datasets such that their elements become one-dimensional tensors of length <font face='courier'>window_length = T + 1</font>.<br>
    <br>
    Initialize <font face='courier'>T</font> and <font face='courier'>window_length</font> as described.
</div>

In [14]:
# YOUR CODE
T = 100
window_length = T + 1

<div class="alert alert-block alert-info">
    Use the method <font face='courier'>tf.data.Dataset.window</font> (see <a href='https://www.tensorflow.org/api_docs/python/tf/data/Dataset#window'>this link</a>) to get items that feature the desired length.<br>
    <br>
    Call the method using the arguments <font face='courier'>shift = 1</font> and <font face='courier'>drop_remainder = True</font>.<br>
    <br>
    The first-mentioned argument <font face='courier'>shift = 1</font> ensures that the first item of the transformed dataset contains the elements <font face='courier'>0,...,window_length - 1</font> of the original dataset, the next item <font face='courier'>1,...,window_length</font>, and so on.<br>
    <br>
    In other words: A window of length <font face='courier'>window_length</font> slides with feed <font face='courier'>shift</font> over the original dataset and extracts sequence by sequence until the end of the dataset is reached.<br>
    <br>
    The latter argument <font face='courier'>drop_remainder = True</font> ensures that no shorter sequences are extracted towards the end of the dataset, where the window could potentially slide beyond the end of the dataset.
</div>

In [15]:
# YOUR CODE
hamlet_1_dataset = hamlet_1_dataset.window(size=window_length,shift = 1, drop_remainder = True)
hamlet_2_dataset = hamlet_2_dataset.window(size=window_length,shift = 1, drop_remainder = True)
hamlet_3_dataset = hamlet_3_dataset.window(size=window_length,shift = 1, drop_remainder = True)

<div class="alert alert-block alert-info">
    Execute the following code.
</div>

In [16]:
for window in hamlet_1_dataset.take(1):
    print(window)
    for item in window.take(10):
        print(item)

<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(20, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


**Checkpoint:** You should have obtained an output like this:

```
<_VariantDataset shapes: (), types: tf.int32>
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(20, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)
```

<div class="alert alert-block alert-info">
    As you can see, the transformed datasets <font face='courier'>hamlet_1_dataset</font>, <font face='courier'>hamlet_2_dataset</font> and <font face='courier'>hamlet_3_dataset</font> have now elements which are again objects of type <font face='courier'>tf.data.Dataset</font> (or of a derived class).<br>
    <br>
    Each of these sub-datasets <font face='courier'>window</font> contains a number of <font face='courier'>window_size</font> single-valued tensors.<br>
    <br>
    Apply the method <font face='courier'>tf.data.Dataset.flat_map</font> (see <a href='https://www.tensorflow.org/api_docs/python/tf/data/Dataset#flat_map'>this link</a>) to all three datasets to transform the sub-datasets <font face='courier'>window</font> into one-dimensional tensors of length <font face='courier'>window_length</font>.<br>
    <br>
    Pass a function which maps <font face='courier'>window</font> to <font face='courier'>window.batch(window_length)</font>.
</div>

In [17]:
# YOUR CODE
hamlet_1_dataset = hamlet_1_dataset.flat_map(lambda window: window.batch(window_length))
hamlet_2_dataset = hamlet_2_dataset.flat_map(lambda window: window.batch(window_length))
hamlet_3_dataset = hamlet_3_dataset.flat_map(lambda window: window.batch(window_length))

<div class="alert alert-block alert-info">
    Display the first element of <font face='courier'>hamlet_1_dataset</font>.
</div>

In [18]:
# YOUR CODE
for i in hamlet_1_dataset.take(1):
    print(i)

tf.Tensor(
[ 2  5  1  0  2  9  4 20  1 13  6  1  0  3 18  0  5  4 14 10  1  2 11 11
  4 19  2 12  8  0 22  9  6 14 12  8 21  0  8 19  3  1  7  4  0 22  9  6
 14  4 21 11 11  1  7  2  1  9  0 23  4  9  7  4  9 13  3  0  4  7 13  0
 18  9  4  7 19  6  8 19  3  0  2 17  3  0 19  1  7  2  6  7  1 10  8 21
 11 11  0 23  4], shape=(101,), dtype=int32)


**Checkpoint:** You should get the following output:

```
tf.Tensor(
[ 2  5  1  0  2  9  4 20  1 13  6  1  0  3 18  0  5  4 14 10  1  2 11 11
  4 19  2 12  8  0 22  9  6 14 12  8 21  0  8 19  3  1  7  4  0 22  9  6
 14  4 21 11 11  1  7  2  1  9  0 23  4  9  7  4  9 13  3  0  4  7 13  0
 18  9  4  7 19  6  8 19  3  0  2 17  3  0 19  1  7  2  6  7  1 10  8 21
 11 11  0 23  4], shape=(101,), dtype=int32)
```

<div class="alert alert-block alert-info">
    Apply the method <font face='courier'>tf.data.Dataset.concatenate</font> (see <a href='https://www.tensorflow.org/api_docs/python/tf/data/Dataset#concatenate'>this link</a>) to merge<br>
    <br>
    <font face='courier'>hamlet_1_dataset</font>, <font face='courier'>hamlet_2_dataset</font> and <font face='courier'>hamlet_3_dataset</font> to a single dataset <font face='courier'>hamlet_dataset</font>.
</div>

In [19]:
# YOUR CODE
hamlet_dataset= (hamlet_1_dataset.concatenate(hamlet_2_dataset)).concatenate(hamlet_3_dataset)

<div class="alert alert-block alert-info">
    Set <font face='courier'>batch_size = 32</font>.<br>
    <br>
    Apply <font face='courier'>tf.data.Dataset.repeat</font> (without argument),<br>
    <br>
    <font face='courier'>tf.data.Dataset.shuffle</font> (with <font face='courier'>buffer_size = 1000</font>)<br>
    <br>
    and finally <font face='courier'>tf.data.Dataset.batch</font> (with <font face='courier'>drop_remainder=True</font>).
    <a href=''></a>
</div>

In [20]:
tf.random.set_seed(0)
# YOUR CODE
batch_size = 32
hamlet_dataset = hamlet_dataset.repeat()
hamlet_dataset = hamlet_dataset.shuffle(buffer_size=10000)
hamlet_dataset = hamlet_dataset.batch(batch_size=batch_size,drop_remainder=True)

<div class="alert alert-block alert-info">
    Our dataset contains now two-dimensional tensors <font face='courier'>window_batch</font> of size <font face='courier'>(32, 101)</font>.<br>
    <br>
    Each slice <font face='courier'>window_batch[i, :]</font> corresponds to a training example.<br>
    <br>
    Here, we still need to subdivide the training examples into inputs and outputs.<br>
    <br>
    Each single encoded character <font face='courier'>window_batch[i, j]</font> (for <font face='courier'>j=0,...,99</font>) is an  input $\mathbf{x}^{<t>(i)}$ in a time step<br>
    <br>
    and the associated output is <font face='courier'>window_batch[i, j + 1]</font> which corresponds to $\mathbf{y}^{<t>(i)}$.<br>
    <br>
    Apply the method <font face='courier'>tf.data.Dataset.map</font> (see <a href='https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map'>this link</a>) to <font face='courier'>hamlet_dataset</font><br>.
    <br>
    Each batch <font face='courier'>window_batch</font> shall be mapped to a tuple of two tensors of size <font face='courier'>(32, 100)</font>.<br>
    <br>
    The <font face='courier'>[i, :]</font>-th slice of the first tensor shall contain the entries <font face='courier'>window_batch[i, 0:100]</font>.<br>
    <br>
    The corresponding slice of the second tensor shall contain <font face='courier'>window_batch[i, 1:101]</font>.
</div>

In [21]:
# YOUR CODE
hamlet_dataset= hamlet_dataset.map(lambda window_batch: (window_batch[:, 0:100], window_batch[:, 1:101]))

<div class="alert alert-block alert-info">
    Execute the following code.
</div>

In [22]:
for window_batch in hamlet_dataset.take(1):
    [x] = tokenizer.sequences_to_texts([window_batch[0][0, :].numpy() + 1])
    [y] = tokenizer.sequences_to_texts([window_batch[1][0, :].numpy() + 1])
    print(x)
    print()
    print(y)

  o f   y o u n g   f o r t i n b r a s , 
 w h o   i m p o t e n t   a n d   b e d r i d ,   s c a r s e l y   h e a r e s 
 o f   t h i s   h i s   n e p h e w e s   p u r p o s e ,   t o   s u p p

o f   y o u n g   f o r t i n b r a s , 
 w h o   i m p o t e n t   a n d   b e d r i d ,   s c a r s e l y   h e a r e s 
 o f   t h i s   h i s   n e p h e w e s   p u r p o s e ,   t o   s u p p r


**Checkpoint:** Your output should look as follows:

```
  o f   y o u n g   f o r t i n b r a s , 
 w h o   i m p o t e n t   a n d   b e d r i d ,   s c a r s e l y   h e a r e s 
 o f   t h i s   h i s   n e p h e w e s   p u r p o s e ,   t o   s u p p

o f   y o u n g   f o r t i n b r a s , 
 w h o   i m p o t e n t   a n d   b e d r i d ,   s c a r s e l y   h e a r e s 
 o f   t h i s   h i s   n e p h e w e s   p u r p o s e ,   t o   s u p p r
```

<div class="alert alert-block alert-info">
    Apply <font face='courier'>tf.data.Dataset.map</font> again to <font face='courier'>hamlet_dataset</font> to map each element <font face='courier'>(X_batch, Y_batch)</font> to a new tuple.<br>
    <br>
    In the new tuple, <font face='courier'>Y_batch</font> shall remain unchanged while <font face='courier'>X_batch</font> undergoes another encoding via <font face='courier'>tf.one_hot</font> (see <a href='https://www.tensorflow.org/api_docs/python/tf/one_hot'>this link</a>).<br>
    <br>
    Find out, which value <font face='courier'>depth</font> you need to pass <font face='courier'>tf.one_hot</font> in addition to <font face='courier'>X_batch</font>.<br>
    <br>
    Hint: You already computed the correct value above.
</div>

In [23]:
# YOUR CODE
hamlet_dataset= hamlet_dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
for i in hamlet_dataset.take(1):
    print(i)

(<tf.Tensor: shape=(32, 100, 42), dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0.

<div class="alert alert-block alert-info">
    Apply the method <font face='courier'>tf.data.Dataset.prefetch</font> (with <font face='courier'>buffer_size = 1</font>) to <font face='courier'>hamlet_dataset</font> to get your dataset ready for training.
</div>

In [24]:
# YOUR CODE
hamlet_dataset= hamlet_dataset.prefetch(buffer_size= 1)

<div class="alert alert-block alert-info">
    Deduce a formula for the number of elements in <font face='courier'>hamlet_dataset</font>.<br>
    <br>
    Display the result and store it in <font face='courier'>steps_per_epoch</font>.
</div>

In [25]:
# YOUR CODE
steps_per_epoch= int(((len(hamlet_1_encoded)+len(hamlet_2_encoded)+len(hamlet_3_encoded))-3*T)/batch_size)
print(steps_per_epoch)

5014


**Checkpoint:** The number of items in the dataset should be as follows:

```
5014
```

<div class="alert alert-block alert-info">
    Use <font face='courier'>keras.models.Sequential</font> to define a <font face='courier'>model</font> featuring<br>
    <br>
    two hidden GRU layers (see <a href='https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU'>this link</a>) with <font face='courier'>128</font> neurons each, as well as<br>
    <br>
    a fully connected output layer (see <a href='https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense'>this link</a>) with <font face='courier'>max_id</font> neurons and <font face='courier'>softmax</font> activation function.<br>
    <br>
    This model corresponds to the RNN displayed in Fig. 1 with an additional hidden GRU layer.<br>
    <br>
    To make the model generate outputs in each time step, the output layer must be enclosed by a <font face='courier'>keras.layers.TimeDistributed</font> wrapper (see <a href='https://www.tensorflow.org/api_docs/python/tf/keras/layers/TimeDistributed'>this link</a>).<br>
    <br>
    Without the layer, the model would generate only a single output in the very last time step (as the one displaye in Fig. 2).<br>
    <br>
    In case of the GRU layers, you should add <font face='courier'>return_sequences = True</font> for the same purpose.<br>
    <br>
    Also recall that the input layer needs an argument <font face='courier'>input_shape=[None, max_id]</font> (where <font face='courier'>None</font> represents the temporal dimension of the input sequence).<br>
    <br>
    During training, we could just as well replace <font face='courier'>None</font> with <font face='courier'>T</font> as all sequences in the training data have identical length.<br>
    <br>
    However, passing <font face='courier'>None</font> ensures that the model will accept arbitrarily long sequences later.
</div>

In [26]:
# YOUR CODE
model=keras.models.Sequential([keras.layers.GRU(128,return_sequences=True, input_shape=[None,max_id]),keras.layers.GRU(128,return_sequences=True),keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation="softmax"))])

<div class="alert alert-block alert-info">
    Compile <font face='courier'>model</font> using <font face='courier'>sparse_categorical_crossentropy</font> and <font face='courier'>adam</font>.
    <a href=''></a>
</div>

In [27]:
# YOUR CODE
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

<div class="alert alert-block alert-info">
    Train <font face='courier'>model</font> for one epoch. Don't forget to pass the <font face='courier'>steps_per_epoch</font> argument.
    <a href=''></a>
</div>

In [28]:
# YOUR CODE
model.fit(hamlet_dataset, epochs=1, steps_per_epoch= steps_per_epoch)

<div class="alert alert-block alert-info">
    You might have observed that even a single epoch takes quite some time.<br>
    <br>
    Create a file <font face='courier'>hamlet_rnn.py</font> and train the model for <font face='courier'>20</font> epochs on the GPU cluster.<br>
    <br>
    In doing so, use a callback<br>
    <br>
    <font face='courier'>keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)</font><br>
    <br>
    (see <a href='https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping'>this link</a>). Save your model as  <font face='courier'>hamlet_model.h5</font> and load it in the following step for further use in this notebook.
</div>

In [29]:
# YOUR CODE
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
history= model.fit(hamlet_dataset, epochs=20, steps_per_epoch= steps_per_epoch,callbacks=[callback])
model.save('hamlet_model.h5')

Epoch 1/20
5014/5014 [==============================] - 89s 15ms/step - loss: 1.2277
Epoch 2/20
5014/5014 [==============================] - 76s 15ms/step - loss: 0.8818
Epoch 3/20
5014/5014 [==============================] - 75s 15ms/step - loss: 0.7692
Epoch 4/20
5014/5014 [==============================] - 78s 15ms/step - loss: 0.6958
Epoch 5/20
5014/5014 [==============================] - 75s 15ms/step - loss: 0.6398
Epoch 6/20
5014/5014 [==============================] - 75s 15ms/step - loss: 0.5959
Epoch 7/20
5014/5014 [==============================] - 75s 15ms/step - loss: 0.5585
Epoch 8/20
5014/5014 [==============================] - 74s 15ms/step - loss: 0.5296
Epoch 9/20
5014/5014 [==============================] - 73s 15ms/step - loss: 0.5021
Epoch 10/20
5014/5014 [==============================] - 73s 15ms/step - loss: 0.4778
Epoch 11/20
5014/5014 [==============================] - 72s 14ms/step - loss: 0.4543
Epoch 12/20
5014/5014 [==============================] - 73s 14

<div class="alert alert-block alert-info">
    Write a function <font face='courier'>preprocess</font> that takes a argument a list <font face='courier'>texts</font> containing a single string.<br>
    <br>
    Inside this function, use <font face='courier'>tokenizer</font> again to encode the string in <font face='courier'>texts</font> as a NumPy-Array <font face='courier'>X</font>.<br>
    <br>
    The return value of <font face='courier'>preprocess</font> shall be the one-hot encoded version of <font face='courier'>X</font>.
</div>

In [30]:
# YOUR CODE
model= tf.keras.models.load_model('hamlet_model.h5')
def preprocess(texts):
    x = tokenizer.texts_to_sequences(texts)
    x = tf.one_hot(x,depth=max_id)
    return x

<div class="alert alert-block alert-info">
    The function <font face='courier'>next_char</font> takes as argument a string <font face='courier'>text</font> and a positive number <font face='courier'>temperature</font><br>
    <br>
    with the goal to generate the next character after <font face='courier'>text</font>.<br>
    <br>
    Replace all placeholders <font face='courier'>None</font> as follows:<br>
    <br>
    First, apply <font face='courier'>preprocess</font> to encode <font face='courier'>text</font> and store the result in <font face='courier'>X_new</font>.<br>
    <br>
    Second, apply <font face='courier'>model.predict</font> the generate outputs given the input sequence <font face='courier'>X_new</font>.<br>
    <br>
    Store the output from the last time step in <font face='courier'>y_proba</font>.<br>
    <br>
    <font face='courier'>rescaled_logits</font> and <font face='courier'>char_id</font> are finally used to generate a one-element sample from  $\{1,\dots,\mathrm{max\_id}\}$.<br>
    <br>
    The generated sample is the encoded version of the character to be generated.<br>
    <br>
    Hence, <font face='courier'>char_id.numpy()</font> will return the character itself.
</div>

In [31]:
def next_char(text, temperature=1):
    X_new = preprocess(text)
    y_proba = model.predict(X_new)
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

next_char(hamlet_1_text)

2005/2005 [==============================] - 4s 2ms/step
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\patel\anaconda3\envs\gpu11.2\lib\site-packages\IPython\core\interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\patel\AppData\Local\Temp\ipykernel_19308\967336109.py", line 8, in <module>
    next_char(hamlet_1_text)
  File "C:\Users\patel\AppData\Local\Temp\ipykernel_19308\967336109.py", line 5, in next_char
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
  File "C:\Users\patel\anaconda3\envs\gpu11.2\lib\site-packages\tensorflow\python\util\traceback_utils.py", line 153, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "C:\Users\patel\anaconda3\envs\gpu11.2\lib\site-packages\tensorflow\python\framework\ops.py", line 7209, in raise_from_not_ok_status
    raise core._status_to_exception(e) from None  # pylint: disable=protected-access
tensorflow.python.framework.errors_impl.InvalidArgumentError: {{function_node __wrapped__Mu

<div class="alert alert-block alert-info">
    The function <font face='courier'>complete_text</font> takes a string <font face='courier'>text</font> and shall append <font face='courier'>n_char</font> subsequent characters generated by your RNN.<br>
    <br>
    Complete the function accordingly. The argument <font face='courier'>temperature</font> just needs to be passed to <font face='courier'>next_char</font>.
</div>

In [ ]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text,temperature)
    return text

<div class="alert alert-block alert-info">
    Starting from the initial string <font face='courier'>'Hamlet'</font>, use <font face='courier'>complete_text</font> to generate a text with <font face='courier'>1000</font> characters.<br>
    <br>
    You can vary the argument <font face='courier'>temperature</font> to modify the distribution from which new characters are drawn.<br>
    <br>
    Values close to zero encourage characters that have a high probability according to the distribution generated by your RNN.<br>
    <br>
    If <font face='courier'>temperature</font> is too high, new characters are drawn according to a uniform distribution on the entire vocabulary, which is not desirable.<br>
    <br>
    You can, for example, try different values between <font face='courier'>0</font> and <font face='courier'>2</font> and evaluate generated texts based on how plausible they appear to you.<br>
    <br>
    Display your generated text.
</div>

In [ ]:
# YOUR CODE
complete_text(hamlet_1_text, n_chars=1000, temperature=1)

<div class="alert alert-block alert-info">
    Submit your completed notebook not later than on January 15th, 2023.<br>
    <br>
    Optional: Train an RNN on a text corpus of your own choice and use your own RNN to generate text.
</div>